In [6]:
import keras
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
import pandas as pd
from preprocessing import load_and_preprocess_data

from sklearn.model_selection import train_test_split

In [3]:
padded_docs, artists_onehot_encoded, vocab_size, max_length = load_and_preprocess_data("spotify_millsongdata.csv")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, artists_onehot_encoded, test_size=0.2, random_state=42)

In [ ]:
# lstm = keras.layers.LSTM(units=artists_onehot_encoded.shape[1])
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size, output_dim = 8, input_length = padded_docs[0].shape[0]))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(8, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(8, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(artists_onehot_encoded.shape[1], activation = 'softmax'))

hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
optimizer = keras.optimizers.Adam()
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

: 

In [ ]:
history = model_lstm.fit(
    X_train,
    y_train,
    validation_split = 0.1,
    epochs = 8,
    batch_size = 512
)


NameError: name 'model_lstm' is not defined

: 